# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
import nltk
import pickle
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# load data from database
engine = create_engine('sqlite:///MyProject.db')
df = pd.read_sql_table('TblHelpRequests', engine)  


X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    """
    This function splits text into words and returns the base of the word
    It will be used when building the ML pipeline later in the code
    The argument is text which is feeding from the message column of the df dataframe
    """
    #Transform everything in lower case
    text = text.lower()
    #Tokenize words
    tokens = nltk.word_tokenize(text)
    #Remove stopwords to avoid potential inconsequential tokens
    stop_words = stopwords.words("english")
    word = [word for word in tokens if word not in stop_words]
    #Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
              
    return lemmatized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
#Split between train and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, random_state=42)
#Train pipeline
pipeline.fit(Xtrain, ytrain)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
def report_scores(model, Xtest, ytest):
    """This function applies the sklearn classification_report
       on the required model (in our case pipeline) and returns 
       the f1-score and the precision of the model using the model
       and the test series as arguments
    """
    
    ypreds = model.predict(Xtest)
    
    for i, col in enumerate(ytest):
        print('------------------------------------------------------\n')
        print('FEATURE NAME: {}\n'.format(col))
        print(classification_report(ytest[col], ypreds[:, i]))

report_scores(pipeline, Xtest, ytest) 

------------------------------------------------------

FEATURE NAME: related

             precision    recall  f1-score   support

          0       0.65      0.33      0.44      1568
          1       0.81      0.94      0.87      4930
          2       1.00      0.13      0.23        47

avg / total       0.77      0.79      0.76      6545

------------------------------------------------------

FEATURE NAME: request

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5411
          1       0.86      0.35      0.50      1134

avg / total       0.88      0.88      0.86      6545

------------------------------------------------------

FEATURE NAME: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6513
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545

------------------------------------------------------

FEATURE NAM

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6024
          1       0.64      0.08      0.15       521

avg / total       0.90      0.92      0.90      6545

------------------------------------------------------

FEATURE NAME: medical_products

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6234
          1       0.67      0.10      0.18       311

avg / total       0.94      0.95      0.94      6545

------------------------------------------------------

FEATURE NAME: search_and_rescue

             precision    recall  f1-score   support

          0       0.97      1.00      0.99      6360
          1       0.50      0.02      0.04       185

avg / total       0.96      0.97      0.96      6545

------------------------------------------------------

FEATURE NAME: security

             precision    recall  f1-score   support

          0       0.98      1.00      0.99  

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {'clf__estimator__n_estimators': [20, 50]}

In [16]:
cv = GridSearchCV(pipeline, parameters)

In [17]:
cv.fit(Xtrain, ytrain)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
#print('========================TEST SET=========================')
report_scores(cv, Xtest, ytest)

------------------------------------------------------

FEATURE NAME: related

             precision    recall  f1-score   support

          0       0.77      0.27      0.40      1568
          1       0.80      0.97      0.88      4930
          2       0.92      0.23      0.37        47

avg / total       0.80      0.80      0.76      6545

------------------------------------------------------

FEATURE NAME: request

             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5411
          1       0.89      0.41      0.56      1134

avg / total       0.89      0.89      0.87      6545

------------------------------------------------------

FEATURE NAME: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6513
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545

------------------------------------------------------

FEATURE NAM

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6024
          1       0.74      0.06      0.10       521

avg / total       0.91      0.92      0.89      6545

------------------------------------------------------

FEATURE NAME: medical_products

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6234
          1       0.63      0.06      0.11       311

avg / total       0.94      0.95      0.94      6545

------------------------------------------------------

FEATURE NAME: search_and_rescue

             precision    recall  f1-score   support

          0       0.97      1.00      0.99      6360
          1       0.67      0.01      0.02       185

avg / total       0.96      0.97      0.96      6545

------------------------------------------------------

FEATURE NAME: security

             precision    recall  f1-score   support

          0       0.98      1.00      0.99  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
pipelineimproved = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

parameters_improved = {'clf__estimator__learning_rate': [0.1, 0.3],
                       'clf__estimator__n_estimators': [100, 200]}

                            
cvimproved = GridSearchCV(pipelineimproved, param_grid=parameters_improved)
cvimproved.fit(Xtrain, ytrain)
                            
report_scores(cvimproved, Xtest, ytest)

------------------------------------------------------

FEATURE NAME: related

             precision    recall  f1-score   support

          0       0.67      0.17      0.27      1568
          1       0.78      0.97      0.87      4930
          2       0.64      0.15      0.24        47

avg / total       0.76      0.78      0.72      6545

------------------------------------------------------

FEATURE NAME: request

             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5411
          1       0.83      0.50      0.63      1134

avg / total       0.89      0.90      0.89      6545

------------------------------------------------------

FEATURE NAME: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6513
          1       0.20      0.03      0.05        32

avg / total       0.99      0.99      0.99      6545

------------------------------------------------------

FEATURE NAM

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6044
          1       0.89      0.50      0.64       501

avg / total       0.95      0.96      0.95      6545

------------------------------------------------------

FEATURE NAME: storm

             precision    recall  f1-score   support

          0       0.95      0.99      0.97      5942
          1       0.82      0.50      0.62       603

avg / total       0.94      0.94      0.94      6545

------------------------------------------------------

FEATURE NAME: fire

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6476
          1       0.62      0.12      0.20        69

avg / total       0.99      0.99      0.99      6545

------------------------------------------------------

FEATURE NAME: earthquake

             precision    recall  f1-score   support

          0       0.98      0.99      0.98      5959
          1  

### 9. Export your model as a pickle file

In [20]:
pickle.dump(cvimproved, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.